In [6]:
from flask import Flask,render_template, request, redirect,url_for,flash
from flask_mysqldb import MySQL
from logging import FileHandler,WARNING
import mysql.connector
import datetime
from datetime import date
def age(date1):
    return int(date.today().year-date1.year)
format = '%Y-%m-%d'
file_handler = FileHandler('errorlog.txt')
file_handler.setLevel(WARNING)

app = Flask(__name__)
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = 'MYSQL'
app.config['MYSQL_DB'] = 'ada'

@app.route('/home')
def home():
    return render_template('home.html')
@app.route('/userhome/<Name>/<Eligibility>')
def userhome(Name, Eligibility):
        return render_template('user-home.html',Name=Name,Eligibility=Eligibility)
@app.route('/editnurse/',methods=['POST','GET'])
def editnurse():
    ID=request.form['Nurse_ID']
    if 'Nurse_Name' not in request.form:
        ID=request.form['Nurse_ID']
        cne = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
        curse = cne.cursor()
        delete_nurse=("DELETE FROM  Nurse WHERE Nurse_ID=%s")
        curse.execute(delete_nurse,(ID,))
        cne.commit()
        curse.close()
        cne.close()
        return render_template('admin.html',listtype='home',success=True)
    else:
        print('Del')
        ID=request.form['Nurse_ID']
        Name=request.form['Nurse_Name']
        cne = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
        curse = cne.cursor()
        add_nurse=("INSERT INTO Nurse(Nurse_Name, Nurse_ID) VALUES (%s,%s)")
        curse.execute(add_nurse,(Name,ID))
        cne.commit()
        curse.close()
        cne.close()
        return render_template('admin.html',listtype='home',success=True)
@app.route('/edituser/',methods=['POST','GET'])
def edituser():
    ID=request.form['AadharID']
    return render_template('edit.html',ID=ID, form='edituser')
@app.route('/admin/',methods=['POST','GET'],defaults= {'listtype':None, 'function':None})
@app.route('/admin/<listtype>',methods=['POST','GET'],defaults={'function':None})
@app.route('/admin/<listtype>/<function>',methods=['POST','GET'])
def admin(listtype,function):
    if request.method=='POST':
         if listtype=='home':
            return render_template('admin.html',listtype=listtype ) 
         else:
            listtype=request.form['listtype']
            if listtype== 'Waitlist':
             cnw = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursw = cnw.cursor()
             query_wait=("SELECT  Name,Aadhaar_ID, Blood_Group FROM  Volunteer WHERE Eligible='Waiting'")
             cursw.execute(query_wait)
             waitlist=cursw.fetchall()
             cursw.close()
             cnw.close()
             return render_template('admin.html',listtype=listtype,function=waitlist)
            elif listtype== 'CheckUp':
             cnd = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursd = cnd.cursor()
             query_donor=("SELECT  Name,Aadhaar_ID, Blood_Group FROM  Volunteer WHERE Eligible='CheckUp'")
             cursd.execute(query_donor)
             donor_list=cursd.fetchall()
             cursd.close()
             cnd.close()
             return render_template('admin.html',listtype=listtype,function=donor_list)
            elif listtype== 'Nurselist':
             cnd = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursd = cnd.cursor()
             query_donor=("SELECT * FROM  Nurse")
             cursd.execute(query_donor)
             donor_list=cursd.fetchall()
             cursd.close()
             cnd.close()
             return render_template('admin.html',listtype=listtype,function=donor_list)

            elif listtype== 'Donor':
             cnd = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursd = cnd.cursor()
             query_donor=("SELECT  Name,Aadhaar_ID, Blood_Group FROM  Volunteer WHERE Eligible='Donor'")
             cursd.execute(query_donor)
             donor_list=cursd.fetchall()
             cursd.close()
             cnd.close()
             return render_template('admin.html',listtype=listtype,function=donor_list)
            elif listtype== 'Donation-list':
             cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursdl = cndl.cursor()
             query_donorl=("SELECT  Name, Aadhaar_ID, Blood_Group FROM  Volunteer WHERE Eligible='Eligible'")
             cursdl.execute(query_donorl)
             donation_list=cursdl.fetchall()
             cursdl.close()
             cndl.close()
             return render_template('admin.html',listtype=listtype,function=donation_list)
            elif listtype=='Appointments':
             cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursdl = cndl.cursor()
             query_donorl=("SELECT  Aadhaar_ID, Nurse_ID, AppDate, Reason FROM  Appointment WHERE Completion=False")
             cursdl.execute(query_donorl)
             donation_list=cursdl.fetchall()
             cursdl.close()
             cndl.close()
             return render_template('admin.html',listtype=listtype,function=donation_list)
            elif listtype=='Stockroom':
             cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursdl = cndl.cursor()
             query_donorl=("SELECT  * FROM  Stockroom")
             cursdl.execute(query_donorl)
             donation_list=cursdl.fetchall()
             cursdl.close()
             cndl.close()
             return render_template('admin.html',listtype=listtype,function=donation_list)
            elif listtype=='BloodBank':
             cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
             cursdl = cndl.cursor()
             query_blood=("SELECT  * FROM  Blood")
             cursdl.execute(query_blood)
             blood_list=cursdl.fetchall()
             cursdl.close()
             cndl.close()
             return render_template('admin.html',listtype=listtype,function=blood_list)
            
            else:
                return redirect(url_for('admin',listtype=listtype))


    else:
        return render_template('admin.html',listtype=listtype)
@app.route('/login',methods=['POST','GET'])
def login():
    error = None
    if request.method == 'POST':
        ID=request.form['AadharID']
        Pass=request.form['password']
        if ID=='admintoor321' and Pass=='admin1234':
            return redirect(url_for('admin',listtype='home'))
        else:
         cnl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
         cursl = cnl.cursor()
         query_user=("SELECT * FROM  Volunteer WHERE Aadhaar_ID=%s")
         cursl.execute(query_user,(ID,))
         log_try=cursl.fetchone()
         query_app=("SELECT AppDate,NULL FROM Appointment WHERE Completion=False AND Aadhaar_ID=%s ORDER BY AppDate ASC")
         cursl.execute(query_app,(ID,))
         Dat=cursl.fetchone()
         if log_try ==None:
             error="Invalid Login Data"
             render_template('login.html',error=error)
         elif log_try[1]==ID:
             return render_template('user-home.html',Name=log_try[0],Eligibility=log_try[12],AppDate=Dat)             
    else:
        return render_template('login.html')
    return render_template('login.html',error=error)
@app.route('/addappoint',methods=['POST','GET'])
def addappoint():
    cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    cursdl = cndl.cursor()
    query_volunteer=("SELECT  Aadhaar_ID, NULL FROM  Volunteer")
    query_nurse=("SELECT  Nurse_ID,NULL FROM  Nurse")
    cursdl.execute(query_volunteer)
    Volun_list=cursdl.fetchall()
    cursdl.execute(query_nurse)
    nurse_list=cursdl.fetchall()
    cursdl.close()
    cndl.close()
    return render_template('edit.html',form='addappoint',user_id=Volun_list, nurse_id=nurse_list)
@app.route('/editappoint',methods=['POST','GET'])
def editappoint():
    cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    cursdl = cndl.cursor()
    A_ID=request.form['AadharID']
    query_nurse=("SELECT  Nurse_ID,NULL FROM  Nurse")
    cursdl.execute(query_nurse)
    nurse_list=cursdl.fetchall()
    cursdl.close()
    cndl.close()
    return render_template('edit.html',form='editappoint',user_id=A_ID, nurse_id=nurse_list)
@app.route('/edit_appoint_commit',methods=['POST','GET'])
def edit_appoint_commit():
    cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    cursdl = cndl.cursor()
    A_ID=request.form['A_ID']
    A_date=request.form['AppDate']
    N_ID=request.form['Nurse_ID']
    Reason=request.form['Reason']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    upd_stmt = ("UPDATE Appointment SET AppDate=%s,Nurse_ID=%s,Reason=%s WHERE Aadhaar_ID=%s AND Completion=False")
    cursr = cnr.cursor()
    cursr.execute(upd_stmt,(A_date, N_ID,Reason, A_ID))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True)
@app.route('/delete_appoint_commit',methods=['POST','GET'])
def delete_appoint_commit():
    A_ID=request.form['A_ID']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    upd_stmt = ("DELETE FROM Appointment WHERE Aadhaar_ID=%s AND Completion=False")
    cursr = cnr.cursor()
    cursr.execute(upd_stmt,(A_ID,))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True)
@app.route('/editstock',methods=['POST','GET'])
def editstock():
    ID=request.form['EquipID']
    return render_template('edit.html',form='editstock',ID=ID )
@app.route('/edit_stock_commit',methods=['POST','GET'])
def edit_stock_commit():
    ID=request.form['ID']
    No=request.form['No']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    edt_stmt = ("UPDATE Stockroom SET Equipment_stock=%s WHERE Equipment_ID=%s")
    cursr = cnr.cursor()
    cursr.execute(edt_stmt,(No,ID))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True)
@app.route('/addstock',methods=['POST','GET'])
def addstock():
    return render_template('edit.html',form='addstock' )
@app.route('/add_stock_commit',methods=['POST','GET']) 
def add_stock_commit():
    ID=request.form['ID']
    Name=request.form['Name']
    No=request.form['No']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    edt_stmt = ("INSERT INTO Stockroom VALUES(%s,%s,%s)")
    cursr = cnr.cursor()
    cursr.execute(edt_stmt,(ID,Name,No))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True)
@app.route('/register',methods=['POST','GET'])
def register():
    error = None
    if request.method == 'POST':
     cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
     ID=request.form['AadharID']
     Name=request.form['Name']
     Password=request.form['password']
     Sex=request.form['customRadioInline1']
     Address=request.form['Address']
     DOB=request.form['DOB']
     Age=age(datetime.datetime.strptime(DOB, format))
     BloodGroup=request.form['BloodGroup']
     insert_stmt = ("INSERT INTO Volunteer (Name,Aadhaar_ID, DOB, Password, Sex, Address, Blood_group, Age,AIDS,HepC, Diabetes, Other, Eligible) VALUES (%s,%s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s)")
     cursr = cnr.cursor()
     cursr.execute(insert_stmt,(Name,ID,DOB,Password,Sex,Address,BloodGroup,Age,False,False,False,False,"Waiting"))
     cnr.commit()
     cursr.close()
     return redirect(url_for('login'))
    else:
        return render_template('register.html')
@app.route('/appoint_commit',methods=['POST','GET'])
def appoint_commit():
    A_ID=request.form['Aadhaar_ID']
    N_ID=request.form['Nurse_ID']
    A_date=request.form['AppDate']
    Reason=request.form['Reason']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    insert_stmt = ("INSERT INTO Appointment (Aadhaar_ID,Nurse_ID, AppDate, Reason,Completion ) VALUES (%s,%s, %s, %s, %s)")
    cursr = cnr.cursor()
    print(A_date)
    cursr.execute(insert_stmt,(A_ID,N_ID,A_date,Reason,False))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True)
@app.route('/user_admin_commit',methods=['POST','GET'])
def user_admin_commit():
    ID=request.form['Aadhaar_ID']
    AIDS=request.form['AIDS']
    HepC=request.form['HepC']
    Diabetes=request.form['Diabetes']
    Other=request.form['Other']
    Eligible=request.form['Eligible']
    cnr = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    edit_stmt=(" UPDATE Volunteer SET AIDS=%s, HepC =%s , Diabetes= %s, Other= %s, Eligible=%s WHERE Aadhaar_ID= %s")
    cursr = cnr.cursor()
    cursr.execute(edit_stmt,(AIDS,HepC,Diabetes,Other,Eligible,ID))
    cnr.commit()
    cursr.close()
    return render_template('admin.html',listtype='home',success=True )
@app.route('/deleteblood',methods=['POST','GET'])
def deleteblood():
    ID=request.form['ID']
    cne = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    curse = cne.cursor()
    delete_blood=("DELETE FROM  Blood WHERE ID=%s")
    curse.execute(delete_blood,(ID,))
    cne.commit()
    curse.close()
    cne.close()
    return render_template('admin.html',listtype='home',success=True )
@app.route('/addblood',methods=['POST','GET'])
def addblood():
    cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    cursdl = cndl.cursor()
    query_volunteer=("SELECT  Aadhaar_ID, NULL FROM  Volunteer WHERE Eligible='Eligible'" )
    query_nurse=("SELECT  Nurse_ID,NULL FROM  Nurse")
    cursdl.execute(query_volunteer)
    Volun_list=cursdl.fetchall()
    cursdl.execute(query_nurse)
    nurse_list=cursdl.fetchall()
    cursdl.close()
    cndl.close()
    return render_template('edit.html',form='addblood',user_id=Volun_list, nurse_id=nurse_list)
@app.route('/blood_commit',methods=['POST','GET'])
def blood_commit():
    A_ID=request.form['Aadhaar_ID']
    N_ID=request.form['Nurse_ID']
    A_date=request.form['AppDate']
    ID=request.form['ID']
    Amount=request.form['Amount']
    cndl = mysql.connector.connect(user='root', password='MYSQL',host='localhost',database='ada')
    cursdl = cndl.cursor()
    query_group=("SELECT  Blood_group,NULL FROM  Volunteer WHERE Aadhaar_ID= %s ")
    cursdl.execute(query_group,(A_ID,))
    Blood_group=cursdl.fetchone()
    print(Blood_group)
    insert_stmt = ("INSERT INTO Blood (ID,Aadhaar_ID,Nurse_ID, Amount,Blood_group,Expiry) VALUES (%s,%s, %s, %s, %s,%s)")
    cursdl.execute(insert_stmt,(ID,A_ID,N_ID,Amount,Blood_group[0],A_date))
    cndl.commit()
    cursdl.close()
    cndl.close()
    return render_template('admin.html',listtype='home',success=True )
if __name__ == '__main__':
    app.run(port=4004)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 49)

True
